## Загрузим нужные библиотеки

Идеи:
~~1) Переменная - количество авторов. ~~

2) OneHotEncoder - авторов в число (автор чувствует статью)

~~3) Отдельно выделить без авторов?~~

4) Определить популярность слов в теге, используя гугл трендс за период --> Интересно, но сложно --> **Парсинг**

5) Определять выходные (например). => Поиграться с датой (мб часы утренние/вечерние)

***6) Заменить категорию, сессию, автора (и/или) на среднее значение по frp (например) (Если много, то взять из этого максимум)*** 

7) Аналогично с тегами (Узнать че за теги (изи) + популярность) -!!! (Траблы с тегами видимости) !!! --> **Парсинг**

8) Распарсить сайт для поиска авторов и посчитать их среднюю популярность тип. (За 10 статей, например) -!!! (Траблы с тегами видимости) !!! --> **Парсинг**

9) Получить длину слов в статье (Влияет как минимум на процент читателей полность прочитавших статью) --> **Парсинг**

~~10) Предсказать на основе View и Depth frp~~

11) Сгруппировать по сессии??

12) Добавить флаг на наличие в статье картинки --> **Парсинг**

13) Курс доллара в этот момент?? --> **Парсинг**

14) Ансабль разных моделей --> ? хз, XGB это по идее уже и есть ансабль. Только ансамбль лесов; Попробовать **другие модели??**

15) Масштабирование данных --> По идее для деревьев нет смысла

In [1]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV


In [2]:
import numpy as np
import xgboost as xg
from sklearn.model_selection import cross_validate

In [26]:
import seaborn as sns
import matplotlib.pyplot as plt

Выполним загрузу датсета

In [112]:
df_train = pd.read_csv("content/train_dataset_train.csv", index_col= 0)
df_test = pd.read_csv("content/test_dataset_test.csv", index_col= 0)
sample = pd.read_csv("content/sample_solution.csv", index_col= 0)

Скидываем csv для Илюхи

In [123]:
#import csv

with open("IDs.csv", "w", newline="") as f:
    #writer = csv.writer(f)
    for ID in list(df_train.index.map(lambda x: x[:24])):
        f.write('\n')
        f.write(ID)
        
    for ID in list(df_test.index.map(lambda x: x[:24])):
        f.write('\n')
        f.write(ID)
        
    #writer.writerows(list(df_train.index.map(lambda x: x[:24])))

Скачиваем новый csv

In [126]:
df_test = pd.read_csv("content/! rbc.csv", sep = '|')

In [127]:
df_test

,id,title,overview,text,article_image,inner_blocksid,title.1,overview.1,text.1,article_image.1,inner_blocks
0,624ac09c9a7947db3d80c98e,Европейский банк развития приостановил доступ ...,В заявлении банка говорится: «Ни в одной из эт...,Совет управляющих Европейского банка реконстру...,1.0,2.0,NaN,NaN,NaN,NaN,NaN
1,620f6b899a7947701cf489e1,Кремль назвал регулярным процессом учебные зап...,"Все страны, которым положено знать об учениях,...",Учения стратегических сил с запусками баллисти...,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,620730cf9a7947ab96a44e27,Госсекретарь Швеции заявила о нежелании вступа...,NaN,Эскалация на Украине не дала повода Швеции вст...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Песков напомнил, что Путин...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Политика,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Министр иностранных дел России Сергей Лавров о...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,&nbsp;,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6262a5889a79470b78c9ca30,Песков назвал прагматичной выдачу лицензии Газ...,NaN,Решение Великобритании выдать Газпромбанку и е...,1.0,2.0,NaN,NaN,NaN,NaN,NaN
8,626678929a79477ca0101568,В Хабаровске задержали главу филиала РАНХиГС п...,NaN,Глава дальневосточного филиала Российской акад...,1.0,2.0,NaN,NaN,NaN,NaN,NaN
9,61f9580f9a794794245a82b9,Глава ВОЗ заявил о росте смертности из-за COVI...,NaN,Всемирная организация здравоохранения обеспоко...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def data_encoding(df_tr, df_tst):
    # Создаем объединенный датафрейм трейна и теста. 
    # Заменим там перемнные и разобьем его обратно
    
    frames = [df_tr[["category", "session", "authors"]], df_tst[["category", "session", "authors"]]]
    df_temp = pd.concat(frames)
    
    # Заменяем категориальные значения на int
    
    df_temp["category"] = df_temp["category"].astype('category')
    df_temp["category"] = df_temp["category"].cat.codes
    df_temp["category"] = df_temp["category"].astype('int')
    
    df_temp["authors"] = df_temp["authors"].astype('category')
    df_temp["authors"] = df_temp["authors"].cat.codes
    df_temp["authors"] = df_temp["authors"].astype('int')
    
    df_temp["session"] = df_temp["session"].astype('category')
    df_temp["session"] = df_temp["session"].cat.codes
    df_temp["session"] = df_temp["session"].astype('int')
    
    #df_temp["tags"] = df_temp["tags"].astype('category')
    #df_temp["tags"] = df_temp["tags"].cat.codes
    #df_temp["tags"] = df_temp["tags"].astype('int')
    
    # Мерджим полученные данные с траэйном и тестом
    
    df_tr = pd.merge(
        df_tr,
        df_temp,
        how="left",
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_code"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_tst = pd.merge(
        df_tst,
        df_temp,
        how="left",
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_code"),
        copy=True,
        indicator=False,
        validate=None,
    )
    #print(df_tr.head(3))
    
    return df_tr, df_tst

In [5]:
def feature_engineering(df_tr, df_tst):
    
    # Добавим число тегов и авторов
    
    for dataset in [df_tr, df_tst]:
        
        dataset['n_authors'] = dataset['authors'].map(lambda x: 0 if x.split(',') == ['[]'] else len(x.split(',')) )
        dataset['n_tags'] = dataset['tags'].map(lambda x: 0 if x.split(',') == ['[]'] else len(x.split(',')) )
        
        # Добавим день, меся, год
        
        dataset['day'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%d").astype(int)
        dataset['mounth'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%m").astype(int)
        dataset['year'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%y").astype(int)
        
    return df_tr, df_tst

In [6]:
df_train, df_test = data_encoding(df_train, df_test)

In [7]:
df_train, df_test = feature_engineering(df_train, df_test)

In [8]:
df_train.head(2)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ ...,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f86...",20460,1.134,35.850,2,2405,739,0,4,4,4,22
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные зап...,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e53...",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f1...",19038,1.142,38.355,0,2724,46,2,3,18,2,22


In [111]:
df_train.tags[0]

"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']"

In [9]:
df_test.head(2)

,title,publish_date,session,authors,ctr,category,tags,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,«Крайне провокационная тема»: как Россия и США...,2022-02-01 17:02:44,J0AvX96vTAaQCiWVbzoMdw,[],0.000,5409f11ce063da9c8b588a12,"['58ca77679a794700b1e37f0a', '5409f1b7e063daa0...",0,2510,739,0,5,1,2,22
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,Власти Херсонской области пообещали сделать ру...,2022-05-24 00:50:55,QS5CqzXYRnmDdR2LaSreEw,[],1.598,5409f11ce063da9c8b588a12,"['5433603acbb20f6e5def0cc5', '5409f420e063daa0...",0,3427,739,0,4,24,5,22


In [11]:
'''df_train.index[0][:24]

import requests
from bs4 import BeautifulSoup

url = 'https://www.rbc.ru/politics/27/05/2022/628f61e09a7947b3726120a5' #+ df_train.index[1][:24]
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

print(soup)

# article__header__counter js-insert-views-count
'''

"df_train.index[0][:24]\n\nimport requests\nfrom bs4 import BeautifulSoup\n\nurl = 'https://www.rbc.ru/politics/27/05/2022/628f61e09a7947b3726120a5' #+ df_train.index[1][:24]\nresponse = requests.get(url)\nsoup = BeautifulSoup(response.text, 'lxml')\n\nprint(soup)\n\n# article__header__counter js-insert-views-count\n"

Разбиваем датасет для тернировки

In [17]:
X = df_train.drop(['title', 'publish_date', 'session', 'authors', 'category', 'tags',
                  'views', 'depth', 'full_reads_percent'], axis = 1)

y = df_train[['views', 'depth', 'full_reads_percent']]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
%%time

model_views = RandomForestRegressor(n_estimators = 150)
scores_v = cross_validate(model_views, 
                          X_train, 
                          y_train['views'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())


views : 0.4258507954522142


In [39]:
'''%%time
parametrs = { 'n_estimators': [50, 70],
              'max_depth': [2, 6, 10],
              'min_samples_leaf': [1, 5, 10],
              'min_samples_split': [2, 5, 10] }

model_1 = RandomForestRegressor()


grid = GridSearchCV(model_1, parametrs, cv=5, scoring='r2')
grid.fit(X_train, y_train['views'])
'''

'''{'max_depth': 6,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 70}'''

"{'max_depth': 6,\n 'min_samples_leaf': 1,\n 'min_samples_split': 5,\n 'n_estimators': 70}"

Выбираем какие фичи убрать

In [44]:
X.columns

Index(['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
       'n_tags', 'day', 'mounth', 'year'],
      dtype='object')

In [49]:
model_1 = RandomForestRegressor(max_depth = 6, min_samples_split = 5, n_estimators = 100)
X_tmp = X.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis = 1)

scores_v = cross_validate(model_1, X_tmp, y['full_reads_percent'], cv=5, scoring='r2')
print('Используя все фичи: ', scores_v['test_score'].mean())

for fiature in ['ctr', 'authors_code', 'day', 'mounth']:
    model_1 = xg.XGBRegressor()
    scores_v = cross_validate(model_1, 
                              X_tmp.drop(fiature, axis = 1),
                              y['full_reads_percent'], cv=5, scoring='r2')
    print('Без ',fiature,':', scores_v['test_score'].mean())

Используя все фичи:  0.27538062185489076
Без  ctr : 0.22614034138390232
Без  authors_code : 0.11185819594385157
Без  day : 0.26260291129654395
Без  mounth : 0.24108976859322243


In [50]:
model_views = xg.XGBRegressor()
model_depth = xg.XGBRegressor()
model_full_reads_percent = RandomForestRegressor(max_depth = 6, min_samples_split = 5, n_estimators = 100)

In [51]:
%%time
model_views.fit(X.drop(['n_tags'] , axis=1), y['views'])
model_depth.fit(X.drop(['session_code'], axis=1), y['depth'])
model_full_reads_percent.fit(X.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis=1), y['full_reads_percent'])

Wall time: 1.7 s


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [54]:
test_df = df_test.drop(["title", "publish_date", "session", "tags", "authors", "category"], axis = 1)

In [55]:
test_df

,ctr,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,0.000,0,2510,739,0,5,1,2,22
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,1.598,0,3427,739,0,4,24,5,22
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,2.006,0,4312,739,0,3,12,5,22
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,0.000,0,7405,739,0,4,19,5,22
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,9.136,0,852,494,1,3,17,2,22
...,...,...,...,...,...,...,...,...,...
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,1.788,4,7695,358,1,4,23,3,22
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,4.427,3,7300,417,1,2,8,4,22
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,0.000,1,7214,739,0,3,14,4,22


In [56]:
sample['views'] = model_views.predict(test_df.drop(['n_tags'], axis=1))
sample['depth'] = model_depth.predict(test_df.drop(['session_code'], axis=1))
sample['full_reads_percent'] = model_full_reads_percent.predict(test_df.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis=1))

In [58]:
sample.head()

,views,depth,full_reads_percent
document_id,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,20325.433594,1.172853,39.447385
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,9841.697266,1.066869,34.929817
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,9595.762695,1.070459,35.461854
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,7280.983398,1.066451,39.846383
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,103764.703125,1.135663,30.032761


In [59]:
sample.to_csv('content/predictions/test_prediction_02_07.csv')

### Скор на кросс валидации:

In [61]:
%%time
model_1 = xg.XGBRegressor()
scores_v = cross_validate(model_1, 
                          X.drop(['n_tags'],axis = 1), 
                          y['views'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())

model_1 = xg.XGBRegressor()
scores_d = cross_validate(model_1, 
                          X.drop(['session_code'],axis = 1), 
                          y['depth'], cv=5, scoring='r2')
print('depth :', scores_d['test_score'].mean())

model_1 = RandomForestRegressor(max_depth = 6, min_samples_split = 5, n_estimators = 100)
scores_frp = cross_validate(model_1, 
                          X.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis = 1 ), 
                          y['full_reads_percent'], cv=5, scoring='r2')
print('full_reads_percent :', scores_frp['test_score'].mean())

print(0.4 * scores_v['test_score'].mean() + 0.3 * scores_d['test_score'].mean() + 0.3 * scores_frp['test_score'].mean())

views : 0.600797813944905
depth : 0.7598758977060135
full_reads_percent : 0.27671405284448347
0.551296110743111
Wall time: 7.13 s


### Score: 0.653667

Стало чуть хуже => RF не сработал

Тестим KNN:

In [82]:
from sklearn.neighbors import KNeighborsRegressor as KNN

In [85]:
model_views = KNN()
scores_v = cross_validate(model_views, 
                          X_train.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis = 1 ), 
                          y_train['full_reads_percent'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())

views : 0.1733693044772779


In [100]:
%%time
# 1) {'leaf_size': 40, 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
# 2) {'leaf_size': 40, 'n_neighbors': 11, 'weights': 'uniform'}

parametrs = { 'n_neighbors': [7, 9, 11],
              'weights': ['uniform', 'distance'],
              'leaf_size': [40, 50, 60] }

model_1 = KNN()


grid = GridSearchCV(model_1, parametrs, cv=5, scoring='r2')
grid.fit(X_train.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis = 1), y_train['views'])

grid.best_params_

Wall time: 3.76 s


{'leaf_size': 40, 'n_neighbors': 9, 'weights': 'uniform'}

In [108]:

scores_v = cross_validate(grid.best_estimator_, 
                          X_train.drop(['session_code', 'n_tags', 'n_authors', 'year', 'category_code'], axis = 1 ), 
                          y_train['full_reads_percent'], cv=5, scoring='r2')
print('views :', scores_v['test_score'].mean())

views : 0.22296483782213677


Результат грузим с RF

## По итогу: XGB справился лучше на обоих валидациях

Код сессии - почти уникальные. Странно, в беседе писали, что они важные

код автора - нужно распарсить и посмотреть вообще сколько их. И есть ли в тесте неизвестные авторы

Аналогично с тегами.

Категории - уникальные вообще говоря (почти)